In [24]:
import matplotlib.pyplot as plt
import math
import random
import json
import numpy as np 
import open3d as o3d
import sklearn.cluster
from sklearn import metrics

file_data = np.fromfile('/home/jidan/Documents/me5413/1_lidar/lidar_data/frame2.pcd.bin', dtype=np.float32)
points = file_data.reshape((-1, 5))[:, :4]
print(np.shape(points))
# Points: (x, y, z, intensity)

def ground_segmentation(data):
    # 作业1
    # 屏蔽开始
    #初始化数据
    idx_segmented = []
    segmented_cloud = []
    iters = 100   #最大迭代次数  000002.bin：10
    sigma = 0.4     #数据和模型之间可接受的最大差值   000002.bin：0.5   000001.bin: 0.2  000000.bin: 0.15  002979.bin：0.15  004443.bin：0.4
    ##最好模型的参数估计和内点数目,平面表达方程为   aX + bY + cZ +D= 0
    best_a = 0
    best_b = 0
    best_c = 0
    best_d = 0
    pretotal = 0 #上一次inline的点数
    #希望的到正确模型的概率
    P = 0.99
    n = len(data)    #点的数目
    outline_ratio = 0.6   #e :outline_ratio   000002.bin：0.6    000001.bin: 0.5  000000.bin: 0.6   002979.bin：0.6
    for i in range(iters):
        ground_cloud = []
        idx_ground = []
        #step1 选择可以估计出模型的最小数据集，对于平面拟合来说，就是三个点
        sample_index = random.sample(range(n),3)    #重数据集中随机选取3个点
        point1 = data[sample_index[0]]
        point2 = data[sample_index[1]]
        point3 = data[sample_index[2]]
        #step2 求解模型
        ##先求解法向量
        point1_2 = (point1-point2)      #向量 poin1 -> point2
        point1_3 = (point1-point3)      #向量 poin1 -> point3
        N = np.cross(point1_3,point1_2)            #向量叉乘求解 平面法向量
        ##slove model 求解模型的a,b,c,d
        a = N[0]
        b = N[1]
        c = N[2]
        d = -N.dot(point1)
        #step3 将所有数据带入模型，计算出“内点”的数目；(累加在一定误差范围内的适合当前迭代推出模型的数据)
        total_inlier = 0
        pointn_1 = (data - point1)    #sample（三点）外的点 与 sample内的三点其中一点 所构成的向量
        distance = abs(pointn_1.dot(N))/ np.linalg.norm(N)     #求距离
        ##使用距离判断inline
        idx_ground = (distance <= sigma)
        total_inlier = np.sum(idx_ground == True)    #统计inline得点数
        ##判断当前的模型是否比之前估算的模型
        if total_inlier > pretotal:                                           #     log(1 - p)
            iters = math.log(1 - P) / math.log(1 - pow(total_inlier / n, 3))  #N = ------------
            pretotal = total_inlier                                               #log(1-[(1-e)**s])
            #获取最好得 abcd 模型参数
            best_a = a
            best_b = b
            best_c = c
            best_d = d

        # 判断是否当前模型已经符合超过 inline_ratio
        if total_inlier > n*(1-outline_ratio):
            break
    print("iters = %f" %iters)
    #提取分割后得点
    idx_segmented = np.logical_not(idx_ground)
    ground_cloud = data[idx_ground]
    segmented_cloud = data[idx_segmented]
    return ground_cloud,segmented_cloud,idx_ground,idx_segmented

def clustering(points, method):
    if method == 'dbscan':
        db = sklearn.cluster.DBSCAN(eps=1.8, min_samples=20).fit(points)
        labels_db = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
        n_noise_ = list(labels_db).count(-1)

        print("Estimated number of clusters: %d" % n_clusters_)
        print("Estimated number of noise points: %d" % n_noise_)
        return labels_db

    if method == 'kmeans':
        kmeans = sklearn.cluster.KMeans(n_clusters=70, random_state=0, n_init="auto").fit(points)
        labels_km = kmeans.labels_
        return labels_km

    if method == 'meanshift':
        # The following bandwidth can be automatically detected using
        bandwidth = sklearn.cluster.estimate_bandwidth(points, quantile=0.1, n_samples=500)

        ms = sklearn.cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
        ms.fit(points)
        labels_ms = ms.labels_
        cluster_centers = ms.cluster_centers_

        labels_unique = np.unique(labels_ms)
        n_clusters_ = len(labels_unique)

        print("number of estimated clusters : %d" % n_clusters_)
        return labels_ms

    if method == 'optics':
        clust = sklearn.cluster.OPTICS(min_samples=10, xi=0.05, min_cluster_size=0.05)

        # Run the fit
        clust.fit(points)
        labels_op = clust.labels_
        return labels_op

#Visualization
def vis(data,label):
    '''
    :param data: n*3的矩阵
    :param label: n*1的矩阵
    :return: 可视化
    '''
    data=data[:,:3]
    labels=np.asarray(label)
    max_label=labels.max()

    # 颜色
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    pt1 = o3d.geometry.PointCloud()
    pt1.points = o3d.utility.Vector3dVector(data.reshape(-1, 3))
    pt1.colors=o3d.utility.Vector3dVector(colors[:, :3])

    o3d.visualization.draw_geometries([pt1],'part of cloud',width=500,height=500)

(34720, 4)


In [62]:
x = points[:, 0]  # x position of point
y = points[:, 1]  # y position of point
z = points[:, 2]  # z position of point
r = points[:, 3]  # reflectance value of point
d = np.sqrt(x ** 2 + y ** 2)  # Map Distance from sensor
degr = np.degrees(np.arctan(z / d))
 
vals = 'height'
if vals == "height":
    col = z
else:
    col = d

#Clustering
ground_cloud,segmented_cloud, index_ground, index_segmented = ground_segmentation(points[:,0:3])
method = 'dbscan'  #Options: 'dbsegmented_cloudan','kmeans','optics','meanshift'
labels = clustering(segmented_cloud, method)

#Visualization
# vis(points[:,0:3],labels)
vis(segmented_cloud,labels)

#Data saving
# points[:,3] = labels
# points = points.tolist()

print(f"points[index_ground] = {(points[index_segmented])}")
tmp = points[index_ground]
tmp[:,3] = -1
points[index_ground] = tmp
print(f"points[index_ground] = {(points[index_segmented])}")
print(f"points[index_segmented][:,3] = {(points[index_segmented][:,3])}")
print(f"labels = {(labels)}")

points[index_segmented][:,3] = labels
print(f"points[index_segmented][:,3] = {(points[index_segmented][:,3])}")

vis(points[:,0:3],points[:,3])
segmented_cloud = np.insert(segmented_cloud,3,np.transpose(labels),axis = 1)

print(f"points[index_segmented][0:10] = {points[index_segmented][0:10]}")
print(f"segmented_cloud[0:10] = {segmented_cloud[0:10]}")

# segmented_cloud[:,3] = labels
segmented_cloud = segmented_cloud.tolist()

with open('lidar_clustering.json', 'w') as f:
    json.dump(segmented_cloud, f)


iters = 47.581142
Estimated number of clusters: 43
Estimated number of noise points: 2034
points[index_ground] = [[-1.7265594e-03 -4.2208382e-01 -1.5841456e-02  1.5000000e+02]
 [-1.3560485e+01 -3.6877424e-01 -6.5695506e-01  1.5000000e+01]
 [-1.6296936e+01 -3.5155106e-01 -4.0475357e-01  9.0000000e+00]
 ...
 [-1.3675551e+01  3.8186893e-02  1.9219806e+00  7.4000000e+01]
 [-1.3668683e+01  4.2942531e-02  2.2456934e+00  7.0000000e+01]
 [-1.3653681e+01  4.7669467e-02  2.5725033e+00  4.2000000e+01]]
points[index_ground] = [[-1.7265594e-03 -4.2208382e-01 -1.5841456e-02  1.5000000e+02]
 [-1.3560485e+01 -3.6877424e-01 -6.5695506e-01  1.5000000e+01]
 [-1.6296936e+01 -3.5155106e-01 -4.0475357e-01  9.0000000e+00]
 ...
 [-1.3675551e+01  3.8186893e-02  1.9219806e+00  7.4000000e+01]
 [-1.3668683e+01  4.2942531e-02  2.2456934e+00  7.0000000e+01]
 [-1.3653681e+01  4.7669467e-02  2.5725033e+00  4.2000000e+01]]
points[index_segmented][:,3] = [150.  15.   9. ...  74.  70.  42.]
labels = [0 1 2 ... 1 1 1]
po